In [341]:
# string = '''python run.py --dataset {dataset} --blackbox_model {blackbox} --explanation_type {etype} --explanation_model {explanation} --n_features 10 --model_type JTT --evaluate_val --seed 1 --output_dir ./output/eff_{blackbox}_{explanation}_{dataset} '''
# for blackbox in ["lr", "xgb", "rf", "nn", "svm_rbf", ]:
#     # for explanation in ["lime", "factor_gam",  "decision_tree","shap_blackbox_preprocessed"]:
#     for explanation in ["lime", "factor_gam",  "decision_tree"]:
#         for dataset in ["adult_cleaned", "compas_balanced", "lsac"]:
#             if explanation == "lime" or explanation == "shap_blackbox_preprocessed":
#                 etype = "local"
#             else:
#                 etype = "global"
#             print(string.format(\
#                 dataset = dataset,
#                 blackbox = blackbox,
#                 etype = etype,
#                 explanation = explanation
#             ))
            

In [342]:
import os
import sys
sys.path.append('../')
import hashlib
import json
from pathlib import Path
import copy

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn import metrics
import warnings
# NB: Warnings occur when computing metrics for groups with
# low sample sizes. In our case, we don't use the metrics 
# for these groups.
warnings.filterwarnings("ignore")

from compute_metrics import Model
from lib.TabularDataset import dataset_params
from lib import TabularDataset
from result_latex_utils import agg_func
# from result_latex_utils import meanPairDiff, agg_func, bold_max


# res_dir = Path('/scratch/ssd001/home/aparna/explanations-subpopulations/output_main')
# res_dir = Path('../output')
res_dir = Path('../output_robust')

In [343]:
# retain column names
retain_cols=False
list(res_dir.glob('**/*.csv'))

[WindowsPath('../output_robust/eff_lr_lime_adult_cleaned/lr_local_lime.csv'),
 WindowsPath('../output_robust/robust_ijtt/lr_local_lime.csv'),
 WindowsPath('../output_robust/robust_jdro/lr_local_lime.csv'),
 WindowsPath('../output_robust/robust_jtt/lr_local_lime.csv')]

In [344]:
df_raw = []
for path in res_dir.glob('**/*.csv'):
    args = json.load((path.parent/'args.json').open('r'))
    # all settings for paper table 1
    #'lime','shap_blackbox_preprocessed',
    # if args['model_type']!='JTT' and  args['explanation_type']=='local' and args['dataset'] in ['adult','compas_balanced','lsac','adult_cleaned','mimic_tab_robust','mimic_tab_fair_ds']:# and args['explanation_model'] in ['lime_test_new','lime_test','lime','shap_blackbox_preprocessed']:
    if True:
        hparams = [i for i in list(args.keys()) if i not in ['seed', 'output_dir']]
        args['hparam_id'] = hashlib.md5(str([args[i] for i in hparams]).encode('utf-8')).hexdigest()
        
        
        res = pd.read_csv(path)
        
        # get performance on test set
        res = res[res.set=='test']

        if res.shape[0]>0:
            dataset = TabularDataset.Dataset(args['dataset'])
            X_train, X_train_expl, X_val_expl, X_test, y_train, y_train_expl, y_val_expl, y_test, g_train, g_train_expl, g_val_expl, g_test = dataset.get_data(
            retain_cols=retain_cols)
            
            # to check that the whole test set is present
            assert res.shape[0]==g_test.shape[0]
            res[dataset_params[args['dataset']].sensitive_attributes] = g_test.values
            assert res[res['blackbox_prob'].isna()].shape[0]==0
            assert res[res['expl_pred'].isna()].shape[0]==0
                            
            if sum(~np.isfinite(res['expl_pred'].values))>0:
                raise ValueError('For dataset {}, model {}'.format(args['dataset'],
                                                                  args['explanation_model']))
                
            # compute metrics over all items in test set
            args['accuracy_all'] = ((res['expl_pred'].values >= 0.5) == res[
                                          'blackbox_pred']).sum() / len(res)
            fpr, tpr, thresholds = roc_curve(res['blackbox_pred'].values, res['expl_pred'].values
                                             , pos_label=1)
            roc = metrics.auc(fpr, tpr)
            args['auroc_all'] = roc
            args['prevalence_all']= res['blackbox_pred'].mean()

            group_names = dataset_params[args['dataset']].sensitive_attributes
            unique_groups = res[group_names].drop_duplicates()
            for grp in group_names:
                for val in res[grp].unique():
                    unique_groups = unique_groups.append({grp: 
                                                          val, **{i: np.nan for i in group_names if i != grp}}, 
                                                         ignore_index = True)

            for group in range(unique_groups.shape[0]):
                group_i = unique_groups.iloc[group].values
                
                mask = ~pd.isnull(group_i)
                curr_group_names = np.array(group_names)[mask]
                curr_group_vals = group_i[mask]
                sel_rows1 = res[(res[np.array(group_names)[mask]].values ==
                       group_i[mask]).all(1)]
                
                # if we have 2 sensitive group columns
                if len(curr_group_names)==2:
                    sel_rows = res[(res[curr_group_names[0]]== curr_group_vals[0])&
                                   (res[curr_group_names[1]]== curr_group_vals[1])]
                elif len(curr_group_names)==1:
                    sel_rows = res[res[curr_group_names[0]]== curr_group_vals[0]]
                else:
                    print('Warning: Default setting!')
                    sel_rows = res
                
                # checking that logic of row selection works 
                assert sel_rows1.equals(sel_rows)
                
                # computing performance on given group
                group_0_val_model = Model(sel_rows['expl_pred'].values, sel_rows[
                                          'blackbox_pred'].values,
                                         sel_rows[
                                          'blackbox_prob'].values)
                all_metrics = group_0_val_model.compute()
                args_group = {
                    **copy.deepcopy(args),
                    **all_metrics
                             }
                # some metadata
                args_group['group'] = str(unique_groups.iloc[group].values)
                args_group['n'] = len(sel_rows)
                args_group['level'] = mask.sum()
                args_group['prevalence'] = sel_rows['blackbox_pred'].mean()
                args_group['pred_prevalence'] = np.mean(sel_rows['expl_pred']>=0.5)
                df_raw.append(args_group)
                # print(args_group)

df_local = pd.DataFrame(df_raw)

In [345]:
# dataset_params[df_local['dataset'][0]].sensitive_attributes

In [346]:
df_local.columns

Index(['dataset', 'blackbox_model', 'explanation_type', 'explanation_model',
       'n_features', 'model_type', 'seed', 'experiment', 'output_dir',
       'ignore_lime_weights', 'evaluate_val', 'max_epochs', 'perturb_sigma',
       'balance_groups', 'balance_labels', 'balance_group_idx',
       'train_grp_clf', 'grp_clf_attr', 'lr', 'C', 'batch_size', 'debug',
       'jtt_lambda', 'jtt_thres', 'joint_dro_alpha', 'groupdro_eta',
       'tree_depth', 'gam_max_iter', 'hparam_id', 'accuracy_all', 'auroc_all',
       'prevalence_all', 'AUROC', 'ACC', 'epsilon', 'group', 'n', 'level',
       'prevalence', 'pred_prevalence'],
      dtype='object')

In [347]:
df_local.group

0       [1. 4.]
1       [1. 2.]
2       [0. 4.]
3       [1. 1.]
4       [0. 2.]
        ...    
63    [nan  4.]
64    [nan  2.]
65    [nan  1.]
66    [nan  3.]
67    [nan  0.]
Name: group, Length: 68, dtype: object

In [365]:
df_local.model_type.unique()

array(['JTT', 'IJTT', 'JointDRO'], dtype=object)

In [349]:
# group settings
list_groups = [
    '[ 0. nan]',
    '[ 1. nan]',
    '[nan  0.]',
    '[nan  1.]',
    '[nan  2.]',
    '[nan  3.]',
    '[nan  4.]'
    
    
]

# the two sensitive groups of interest for all 4 datasets
# NB: first item in tuple is sex, second is race
# so here sex is considered the sensitive attribute for
# Adult and MIMIC and race for COMPAS and LSAC
list_groups_dict={}
list_groups_dict['adult']=[
    '[ 0. nan]',
    '[ 1. nan]']
list_groups_dict['adult_cleaned']=[
    '[ 0. nan]',
    '[ 1. nan]']
list_groups_dict['mimic_tab_robust']=[
     '[ 0. nan]',
     '[ 1. nan]']

list_groups_dict['lsac']=[
    '[nan  0.]',
    '[nan  1.]',
    '[nan  2.]',
    '[nan  3.]',
    '[nan  4.]']
list_groups_dict['lsac_cat']=[
    '[nan  0.]',
    '[nan  1.]',
    '[nan  2.]',
    '[nan  3.]',
    '[nan  4.]']

list_groups_dict['compas_balanced']=[
    '[nan  0.]',
    '[nan  1.]']



In [350]:
df_alg = []

In [351]:
# fetching hyperparams
for path in res_dir.glob('**/*.csv'):
    args = json.load((path.parent/'args.json').open('r'))
    hparams = [i for i in list(args.keys()) if i not in  ['seed', 'output_dir']]
    break

In [352]:
def get_max_gap_row(row, disp='auroc',kind='mean',return_type='str_sum',
                    gap_or_not='_gap',
                    list_groups=list_groups):
    col_names = []
    col_mean_names = []
    for group in list_groups:
        col_names.append(f'{disp}_{group}{gap_or_not}')
        col_mean_names.append((f'{disp}_{group}{gap_or_not}',kind))
    
    if return_type=='str_sum':
        max_val_ind = np.nanargmax(row[col_mean_names].values)
        val=row[col_names[max_val_ind]]
    elif return_type=='n':
        max_val_ind = np.nanargmax(row[col_mean_names].values)
        val= row[col_mean_names[max_val_ind]]
    else:
        max_val_ind = np.nanargmax(row[col_names].values)
        val= row[col_names[max_val_ind]]
    return val

In [353]:
sens_feature_dict={}
sens_feature_dict['adult_cleaned']='1'
sens_feature_dict['lsac']='2'
sens_feature_dict['mimic_tab_robust']='1'
sens_feature_dict['compas_balanced']='2'


In [354]:
# balance sensitive attribute groups by oversampling during training?
balance_group_ind = False

# balance labels by oversampling during training?
# NB: We will always do this for LSAC, see Appendix
# in paper
balance_labels_ind = False

In [355]:
df_sel=pd.DataFrame(df_raw)
lsac_mean = df_sel[df_sel['dataset'] == 'lsac'].mean()
for key in list(lsac_mean.keys()):
    df_sel[key] = df_sel[key].fillna(lsac_mean[key])
# df_sel=df_sel[((df_sel.dataset!='lsac')&(df_sel.balance_labels==balance_labels_ind))|
#             #   ((df_sel.dataset=='lsac')&(df_sel.balance_labels==True))
#               ((df_sel.dataset=='lsac')&(df_sel.balance_labels==False))
#              ]
df_sel=df_sel[df_sel.balance_groups==balance_group_ind]
df_sel

,dataset,blackbox_model,explanation_type,explanation_model,n_features,model_type,seed,experiment,output_dir,ignore_lime_weights,...,auroc_all,prevalence_all,AUROC,ACC,epsilon,group,n,level,prevalence,pred_prevalence
0,adult_cleaned,lr,local,lime,10,JTT,1,,output\eff_lr_lime_adult_cleaned,False,...,0.942752,0.188946,0.941252,0.783150,-0.082286,[1. 4.],2813,2,0.257732,0.040882
1,adult_cleaned,lr,local,lime,10,JTT,1,,output\eff_lr_lime_adult_cleaned,False,...,0.942752,0.188946,0.950840,0.879121,-0.179015,[1. 2.],273,2,0.139194,0.025641
2,adult_cleaned,lr,local,lime,10,JTT,1,,output\eff_lr_lime_adult_cleaned,False,...,0.942752,0.188946,0.957388,0.938636,-0.242103,[0. 4.],1320,2,0.077273,0.015909
3,adult_cleaned,lr,local,lime,10,JTT,1,,output\eff_lr_lime_adult_cleaned,False,...,0.942752,0.188946,0.896774,0.711340,-0.041104,[1. 1.],97,2,0.360825,0.072165
4,adult_cleaned,lr,local,lime,10,JTT,1,,output\eff_lr_lime_adult_cleaned,False,...,0.942752,0.188946,0.967275,0.970954,-0.285672,[0. 2.],241,2,0.033195,0.004149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,adult_cleaned,lr,local,lime,10,JTT,1,,output\robust_jtt,False,...,0.942048,0.188946,0.941735,0.832567,-0.133111,[nan 4.],4133,1,0.200097,0.032664
64,adult_cleaned,lr,local,lime,10,JTT,1,,output\robust_jtt,False,...,0.942048,0.188946,0.947928,0.920233,-0.229226,[nan 2.],514,1,0.089494,0.017510
65,adult_cleaned,lr,local,lime,10,JTT,1,,output\robust_jtt,False,...,0.942048,0.188946,0.939920,0.791946,-0.109621,[nan 1.],149,1,0.281879,0.073826
66,adult_cleaned,lr,local,lime,10,JTT,1,,output\robust_jtt,False,...,0.942048,0.188946,0.948052,0.882353,-0.208897,[nan 3.],51,1,0.137255,0.019608


In [356]:
LEVEL = 1
df=df_sel
# df=df[df.seed.isin([1,2,3,4,5])]
# df=df[(df.experiment.isin(['lime_balanced_correct_replication']))]

res_df = df.query(f'level == {LEVEL}').groupby('output_dir').apply(agg_func)

metrics = [i for i in res_df.columns if not i.startswith('worst_group')]
out1 = (res_df
     .reset_index()
     .merge(df[hparams + ['output_dir', 'seed', 'hparam_id']].drop_duplicates()))

out2 = (out1.groupby('hparam_id').agg({
   i: ('mean', 'std') for i in metrics
})
       .merge(df[hparams + ['hparam_id']].drop_duplicates().set_index('hparam_id'), left_index = True, right_index = True)
       .reset_index())

for col in metrics:
    # out2[col] = out2[(col, 'mean')].apply(lambda x: '{0:.001%}'.format(x)) +' ± ' +  out2[(col, 'std')].apply(lambda x: '{0:.001%}'.format(x))
    out2[col] = out2[(col, 'mean')].apply(lambda x: '{0:.001%}'.format(x))


out_all=out2.copy()

In [357]:
out_all.to_csv("temp.csv")

In [358]:
print(out_all.isna().sum())

hparam_id                0
(accuracy_all, mean)     0
(accuracy_all, std)      2
(accuracy_min, mean)     0
(accuracy_min, std)      2
                        ..
epsilon_[nan  0.]_gap    0
epsilon_[nan  1.]_gap    0
epsilon_[nan  2.]_gap    0
epsilon_[nan  3.]_gap    0
epsilon_[nan  4.]_gap    0
Length: 195, dtype: int64


In [359]:
auroc_gaps=[]
acc_gaps=[]
for sig in np.sort(out_all.perturb_sigma.unique()):
    temp1 = out_all
    # temp1=out_all[(out_all.explanation_type=='local')&
    #               (out_all.blackbox_model.isin(['lr','nn']))&
    #              (out_all.perturb_sigma==sig)]
    
    # temp1=temp1[temp1.n_features.isin([100])]
    # temp1=temp1[(temp1.explanation_model=='shap_blackbox_preprocessed')|
    #            (temp1.explanation_model=='lime')|
    #            (temp1.explanation_model=='lime_test')|
    #             (temp1.explanation_model=='lime_test_new')
    #            ]
    # temp1.loc[temp1.explanation_model=='shap_blackbox_preprocessed','explanation_model']='SHAP'
    # temp1.loc[temp1.explanation_model=='lime','explanation_model']='LIME'
    # temp1 = temp1[temp1.dataset.isin(['adult','mimic_tab_robust','lsac','adult_cleaned','compas_balanced'])]
    # temp1 = temp1[temp1.explanation_model.isin(['LIME','SHAP'])]

    for perf_metric in ['auroc','accuracy','epsilon']:
        temp1[f'{perf_metric}_max_gap']=temp1.apply(lambda row: get_max_gap_row(row,perf_metric,
                                                                       list_groups=list_groups_dict[row['dataset']]), axis=1)
        temp1[f'{perf_metric}_sens_gap']=temp1.apply(lambda row: row[f'{perf_metric}_sens{sens_feature_dict[row.dataset]}_gap'],axis=1)
        
print(temp1)


                          hparam_id  (accuracy_all, mean)  \
0  2cef927231ffee39fae33dacee461442              0.835005   
1  a01bc04600b589cc9d0515b9f02d3191              0.609826   
2  fdc93bbcf478c07466f2bcc90a281a34              0.842068   

   (accuracy_all, std)  (accuracy_min, mean)  (accuracy_min, std)  \
0                  NaN              0.778523                  NaN   
1                  NaN              0.526316                  NaN   
2             0.000145              0.787858              0.00371   

   (accuracy_minority, mean)  (accuracy_minority, std)  \
0                   0.947368                       NaN   
1                   0.526316                       NaN   
2                   0.947368                       0.0   

   (accuracy_majority, mean)  (accuracy_majority, std)  (accuracy_gap, mean)  \
0                   0.824825                       NaN              0.056482   
1                   0.609485                       NaN              0.083510   
2    

In [360]:
metrics_reported_table1=['auroc_all',
                 'accuracy_max_gap',
                  'auroc_sens_gap',
                 'accuracy_sens_gap',
                 'epsilon_sens_gap'
                      ]
# temp1=temp1[temp1.explanation_model=='LIME']
res = pd.pivot_table(data=temp1[['dataset','blackbox_model','explanation_model']+metrics_reported_table1],
               values=metrics_reported_table1,  index = ['dataset', 'blackbox_model','explanation_model'], 
                                                    aggfunc = lambda x: x,
                                                    )
res

,,,accuracy_max_gap,accuracy_sens_gap,auroc_all,auroc_sens_gap,epsilon_sens_gap
dataset,blackbox_model,explanation_model,,,,,
adult_cleaned,lr,lime,[],[],[],[],[]


In [361]:
# temp1=temp1[temp1.explanation_model.isin(['LIME'])]
print(temp1[['dataset','blackbox_model','explanation_model',
             'auroc_all',
             'auroc_max_gap',
             'accuracy_max_gap',
             'auroc_sens_gap',
             'accuracy_sens_gap',
             'epsilon_sens_gap',
             'model_type'
             ]].sort_values(['dataset','blackbox_model','explanation_model']).to_latex())
# print(res.to_latex())


\begin{tabular}{lllllllllll}
\toprule
{} &        dataset & blackbox\_model & explanation\_model & auroc\_all & auroc\_max\_gap & accuracy\_max\_gap & auroc\_sens\_gap & accuracy\_sens\_gap & epsilon\_sens\_gap & model\_type \\
\midrule
0 &  adult\_cleaned &             lr &              lime &     92.7\% &          0.3\% &             5.4\% &           2.5\% &             16.0\% &            16.2\% &       IJTT \\
1 &  adult\_cleaned &             lr &              lime &     60.1\% &          1.6\% &             1.8\% &           6.3\% &              5.3\% &            15.6\% &   JointDRO \\
2 &  adult\_cleaned &             lr &              lime &     94.2\% &          0.4\% &             5.1\% &           2.1\% &             15.2\% &            15.8\% &        JTT \\
\bottomrule
\end{tabular}



In [367]:
temp1[['dataset','blackbox_model','explanation_model',
             'auroc_all',
             'auroc_max_gap',
             'accuracy_max_gap',
             'auroc_sens_gap',
             'accuracy_sens_gap',
             'epsilon_sens_gap',
             'model_type'
             ]].sort_values(['dataset','blackbox_model','explanation_model'])

,dataset,blackbox_model,explanation_model,auroc_all,auroc_max_gap,accuracy_max_gap,auroc_sens_gap,accuracy_sens_gap,epsilon_sens_gap,model_type
0,adult_cleaned,lr,lime,92.7%,0.3%,5.4%,2.5%,16.0%,16.2%,IJTT
1,adult_cleaned,lr,lime,60.1%,1.6%,1.8%,6.3%,5.3%,15.6%,JointDRO
2,adult_cleaned,lr,lime,94.2%,0.4%,5.1%,2.1%,15.2%,15.8%,JTT
